# Group Assignment 1

## Implementation of Game of Life with(out) DASK
The cell below contains the implementation of the Game of Life with and without DASK. The detailed description of the implementation is given seperately in read.me.

In [ ]:
import numpy as np
import dask
import dask.array as da
from dask.distributed import Client

# Read the grid in the text file
def load_grid_from_file(input_file):
    with open(input_file) as f:
        width, height = map(int, f.readline().split())
        # Make a grid with size of height and width
        grid = []
        for r in range(height):
            grid.append([0] * width)
        # Read the index into the grid
        for line in f: 
            value= line.split() 
            if len(value) >= 2:
                y, x = map(int, value[:2])
                grid[y][x] = 1
            else:
                print("Line doesn't contain two values, check for floating lines at the bottom of file")
        grid = np.array(grid)
    return grid

# save the grid to the text file
def save_grid_to_file(output_file, grid):
    with open(output_file, "w") as f:
        height = len(grid)
        width = len(grid[0])
        f.write(f"{width} {height}\n")
        for y, row in enumerate(grid):
            for x, cell in enumerate(row):
                if cell:
                    f.write(f"{y} {x}\n")

# to calculate the alive neighbors
def count_live_neighbors(grid, x, y, width, height):
    # the neighbor matrix
    neighbors = np.array([
        (-1, -1), (-1, 0), (-1, 1),
        (0, -1),           (0, 1),
        (1, -1),  (1, 0),  (1, 1)
    ])
    # initialize the count
    count = 0
    # loop all possible neighbors
    for dx, dy in neighbors:
        nx, ny = x + dx, y + dy
        # if its neighbor doesn't go beyond the boundary and is alive 
        if 0 <= nx < width and 0 <= ny < height and grid[ny][nx]==1:
            count += 1

    return count

# update the gird
def update_grid(grid):
    # Get height and width
    height = len(grid)
    width = len(grid[0])
    new_grid = np.array([[0] * width for _ in range(height)])

    #apply the rule
    for y in range(height):
        for x in range(width):
            live_neighbors = count_live_neighbors(grid, x, y, width, height)

            if grid[y][x] == 1:
                # Live cell
                if live_neighbors == 2 or live_neighbors == 3:
                    new_grid[y][x] = 1
            else:
                # Dead cell
                if live_neighbors == 3:
                    new_grid[y][x] = 1

    return new_grid

# run game with DASK
def run_game(input_name, output_name, generations, chunksize):
    # Load the grid from the file
    grid = load_grid_from_file(input_name)
    # Dask compute
    grid_da = da.from_array(grid, chunks=(chunksize[0], chunksize[1]))
    for _ in range(generations):   
        grid_da = grid_da.map_overlap(update_grid, depth=1, boundary="none")
    grid = grid_da.compute()

    save_grid_to_file(output_name, grid)
    return grid

# run the game without DASK
def run_game_no_dask(input_name, output_name, generations):
    grid = load_grid_from_file(input_name)
    for _ in range(generations):
        grid = update_grid(grid)
    save_grid_to_file(output_name, grid)
    return grid

This cell contains the main function of the implementation with DASK. If you want to run the came with DASK, please specify all the paramters here.

In [ ]:
## main function with DASK
dask.config.set(scheduler='processes')
# client=Client()
if __name__ == "__main__":
    input_name = "test_data/1000x1000.txt"
    output_name = "test_data/output.txt"
    generations = 2
    chunksize = (500, 500)
    grid = run_game(input_name, output_name, generations, chunksize)
# client.shutdown()
print(grid)

This cell contains the main function of the implementation without DASK. If you want to run the came with DASK, please specify all the paramters here.

In [ ]:
## main function without DASK
if __name__ == "__main__":
    input_name = "test_data/1000x1000.txt"
    output_name = "test_data/output.txt"
    generations = 2
    grid = run_game_no_dask(input_name, output_name, generations)
print(grid)

## Benchmarking and Comparison  
One of objectives of this assignment was to compare the difference between non-parallel and parallel implementation of Conway's Game of Life. In this process we learned more about how different parameters and computational structures affect processing. For this we ran game_dask.py and game_no_dask.py using three different profilers: memory profiler, cProfiler and line_profiler. In addition we ran a comparative analaysis of game_dask.py by looking at how grid sizes, schedulers, and chunk size impacted preformance. 


### Selection of Parameters for Benchmarking and Comparison
The following table provides information on the time taken by the different schedulers to do the computation for different chunk sizes. For the assignment, chunk size 100 was selected as it performed the best as compared to chunk sizes 50 and 10. 

Threads and synchronous provide take the most time. Although threads introduces very little overhead, the reason for it taking more time could be Python’s global interpreter lock (GIL) lock as the code is entirely python-based. It can be seen that ‘processes’ performs the fastest for the computation, possible because it is able to bypass the issues with GIL and provide parallelism. It can also be seen that distributed scheduler takes more time than processes. This could be because distributed scheduler introduces more overhead due to communication and transfer between different processes. 

![timing_table.png](Image/timing_table.png)


#### cProfiler
The first image is the cProfiler for the game_no_dask.py and the second image is the cProfiler for the game_dask.py, with the scheduler being processer. Game_no_dask.py has several large functions that take up most of the run time, whereas game_dask.py seems to have more overhead. This is also reflected in the time it takes to run through one tick. It took game_no_dask.py 14.5s and game_dask.py 26.6s. 

![no_dask_c.png](Image/no_dask_c.png)

![dask_c.png](Image/dask_c.png)

When we increased the gerneations/ticks to 4 we see that the general time pattern is the same but more dramatic. The first image below is the game_no_dask.py ran again and the second image is game_dask.py. We see that a large function still takes up most of the time, the smaller functions all but disapear for game_no_dask.py and have relatively less present for game_dask.py. One notable change is that with more generations game_dask.py produced a faster result. 

![nodask_c_4.png](Image/nodask_c_4.png)

![dask_c_4.png](Image/dask_c_4.png)


#### Kern Line Profiler (kernprof) 
The first image below is the results of the line profiler looking at key functions in game_no_dask.py and the second image below is the results of the line profiler looking at key functions in game_dask.py. Most note worth is the fact that we can see in the game_no_dask.py the lines related to neighbor comparison are ran once for each cell in the grid.  In comparison to game_dask.py, of the main functions no line was ran more than nine times.  The function for comparing neighbors was reviewed once and then reviewing the count of neighboring cells happened 8 times, for all the possible neighbors for a grid cell.  

![lineP_nodask.png](Image/lineP_nodask.png)

![lineP_dask.png](Image/lineP_dask.png)


### Dask Dashboard Observations 
The progress bar below shows the progress of every individual task, namely- getitem, trim, block-info, overlap and array. The check grey section are the tasks that are ready to be run but are not running (if we have more cores, they would be running). The solid colored piece to the left are the completed tasks sitting in the memory and the right-most colors represent the completed tasks that have been realsed from the memory.

![key.jpg](Image/key.jpg)


The task stream below shows the activity of every thread or core across our cluster over time. Every rectangle corresponds to one task. The white spaces correspond to the dead time and the red tasks indicate that there is some communication happening with other tasks that are present in the stream.

![dask_stream.png](Image/dask_stream.png)

The two task streams show that tasks in the bigger grid had less inter-communication (the red rectangles) as compared to the tasks from smaller grid. This could possibly be as big data allows for more parallelism as compared to smaller data size. This could also be because smaller data results in smaller tasks leading to more overhead in terms of scheduling and communication. Big data is more efficiently distributed across different workers, reducing communication overhead. Reference link- https://docs.dask.org/en/latest/dashboard.html


In the task stream we see information about one task but cannot know what is happening within the task and python function. The profile shows the functions (as horizontal bars) within each task along with the other functions that depend and follow other functions. However, only unless you hover over each bar, would you know the function, which is why they are not included here. The graphs below show the activity overtime for the two different grid sizes, bigger size grid having more dynamic progress as compared to smaller grid size.

![activity_over_time_diff_grid.png](Image/activity_over_time_diff_grid.png)
